In [1]:
!pip -q install langchain transformers
!pip -q install accelerate bitsandbytes sentencepiece Xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 112.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 85.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.0/167.0 MB 10.0 MB/s eta 0:00:00


## StarChatAlpha




In [2]:
from huggingface_hub import notebook_login
notebook_login()

## Load the model


In [3]:
import torch
import transformers
from transformers import GenerationConfig, pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("HuggingFaceH4/starchat-alpha")

model = AutoModelForCausalLM.from_pretrained("HuggingFaceH4/starchat-alpha",
                                              load_in_8bit=True,
                                              device_map='auto',
                                              torch_dtype=torch.float16,
                                             use_auth_token= True
                                              )

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:479: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [4]:
tokenizer.eos_token, tokenizer.pad_token

Using pad_token, but it is not set yet.


('<|endoftext|>', None)

In [5]:
tokenizer.eos_token_id, tokenizer.pad_token_id

(0, None)

In [6]:
tokenizer.pad_token_id = 0

### Basic Text generation

In [8]:
full_prompt

NameError: ignored

In [43]:
import textwrap

def generate_response(input_prompt):
    system_prompt = "<|system|>\nBelow is a conversation between a human user and a helpful AI coding assistant.<|end|>\n"

    user_prompt = f"<|user|>\n{input_prompt}<|end|>\n"

    assistant_prompt = "<|assistant|>"

    full_prompt = system_prompt + user_prompt + assistant_prompt

    inputs = tokenizer.encode(full_prompt, return_tensors="pt").to('cuda')
    outputs = model.generate(inputs,
                            eos_token_id = 0,
                            pad_token_id = 0,
                            max_length=1024,
                            early_stopping=True)
    output =  tokenizer.decode(outputs[0])
    output = output[len(full_prompt):]
    if "<|end|>" in output:
        cutoff = output.find("<|end|>")
        output = output[:cutoff]
    #print(input_prompt+'\n')
    # wrapped_text = textwrap.fill(output, width=100)
    # print(wrapped_text +'\n\n')
    return output


In [10]:
generate_response("Write a python function that reverses each word in an input string")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:399: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


Write a python function that reverses each word in an input string


Here's a Python function that reverses each word in a string:

```python
def reverse_words(input_str):
    words = input_str.split()
    reversed_words = [word[::-1] for word in words]
    return " ".join(reversed_words)
```

This function takes an input string and splits it into a list of words using the split() method. It then uses a list comprehension to reverse each word in the list using the [::-1] slicing notation. Finally, the reversed words are joined together using the join() method and the result is returned.


In [44]:
code=['move 3 to nroutes.\n\nmove  to routing-pattern(1).\nmove indexweb to routing-destiny(1).\n\nmove /showsum/%value1/%value2" to routing-pattern(2).\nmove "showsum" to routing-destiny(2).\n\nmove "/showname/%value" to routing-pattern(3).\nmove "showname" to routing-destiny(3).\n',
      '\n\n identification division.\n program-id. indexweb.\n\n data division.\n working-storage section.\n\n 01 the-vars.\n\n 03 COW-vars OCCURS 99 times.\n \n 05 COW-varname pic x(99).\n 05 COW-varvalue pic x(99). \n\n\n procedure division.\n\n call cowtemplate using the-vars "index.cow".\n\n \n goback.\n\n end program indexweb.\n\n',
      '\n\n identification division.\n program-id. showname.\n\n data division.\n working-storage section.\n\n 01 the-vars.\n\n 03 COW-vars OCCURS 99 times.\n \n 05 COW-varname pic x(99).\n 05 COW-varvalue pic x(99). \n\n linkage section.\n\n 01 the-values.\n\n 05 COW-query-values occurs 10 times.\n 10 COW-query-value-name pic x(90).\n 10 COW-query-value pic x(90).\n\n\n procedure division using the-values.\n\n\n MOVE "username" to COW-varname(1).\n MOVE COW-query-value(1) to COW-varvalue(1). \n\n call cowtemplate using the-vars "hello.cow".\n\n \n goback.\n\n end program showname.\n\n']
data = []

# empty the dataframe
#df = pd.DataFrame(columns = ['Code','Documentation'])
for i in code:
  my_dict = {}
  input_code='Write a well detailed documention and itemize what each line of code is doing -- ' + i
  output=generate_response(input_code)
  print(output)
  my_dict['Code'] = i
  my_dict['Documentation'] = output
  data.append(my_dict)
  #df = df.append({'Code':item['name'], 'price':item['price']}, ignore_index = True)


Sure, here's a well-detailed documentation of the code you provided:

The first line of code is a move command that moves a file from the user's Downloads folder to a folder named "routing-pattern" with the number 1.
The second line of code is a move command that moves a file from the user's Downloads folder to a folder named "routing-destiny" with the number 1.
The third line of code is a move command that moves a file from the user's Downloads folder to a folder named "routing-pattern" with the number 2.
The fourth line of code is a move command that moves a file from the user's Downloads folder to a folder named "routing-destiny" with the number 2.
The fifth line of code is a move command that moves a file from the user's Downloads folder to a folder named "routing-pattern" with the number 3.
The sixth line of code is a move command that moves a file from the user's Downloads folder to a folder named "routing-destiny" with the number 3.

In this way, the code creates three folders 

In [45]:
data

[{'Code': 'move 3 to nroutes.\n\nmove  to routing-pattern(1).\nmove indexweb to routing-destiny(1).\n\nmove /showsum/%value1/%value2" to routing-pattern(2).\nmove "showsum" to routing-destiny(2).\n\nmove "/showname/%value" to routing-pattern(3).\nmove "showname" to routing-destiny(3).\n',
  'Documentation': '\nSure, here\'s a well-detailed documentation of the code you provided:\n\nThe first line of code is a move command that moves a file from the user\'s Downloads folder to a folder named "routing-pattern" with the number 1.\nThe second line of code is a move command that moves a file from the user\'s Downloads folder to a folder named "routing-destiny" with the number 1.\nThe third line of code is a move command that moves a file from the user\'s Downloads folder to a folder named "routing-pattern" with the number 2.\nThe fourth line of code is a move command that moves a file from the user\'s Downloads folder to a folder named "routing-destiny" with the number 2.\nThe fifth line of

In [46]:
pd.set_option('display.max_colwidth', None)
pd.DataFrame(data)

,Code,Documentation
0,"move 3 to nroutes.\n\nmove to routing-pattern(1).\nmove indexweb to routing-destiny(1).\n\nmove /showsum/%value1/%value2"" to routing-pattern(2).\nmove ""showsum"" to routing-destiny(2).\n\nmove ""/showname/%value"" to routing-pattern(3).\nmove ""showname"" to routing-destiny(3).\n","\nSure, here's a well-detailed documentation of the code you provided:\n\nThe first line of code is a move command that moves a file from the user's Downloads folder to a folder named ""routing-pattern"" with the number 1.\nThe second line of code is a move command that moves a file from the user's Downloads folder to a folder named ""routing-destiny"" with the number 1.\nThe third line of code is a move command that moves a file from the user's Downloads folder to a folder named ""routing-pattern"" with the number 2.\nThe fourth line of code is a move command that moves a file from the user's Downloads folder to a folder named ""routing-destiny"" with the number 2.\nThe fifth line of code is a move command that moves a file from the user's Downloads folder to a folder named ""routing-pattern"" with the number 3.\nThe sixth line of code is a move command that moves a file from the user's Downloads folder to a folder named ""routing-destiny"" with the number 3.\n\nIn this way, the code creates three folders in the user's Downloads folder, each containing a file named ""showsum"" or ""showname"". The files are moved to these folders based on the values of the variables %value1, %value2, and %value3."
1,"\n\n identification division.\n program-id. indexweb.\n\n data division.\n working-storage section.\n\n 01 the-vars.\n\n 03 COW-vars OCCURS 99 times.\n \n 05 COW-varname pic x(99).\n 05 COW-varvalue pic x(99). \n\n\n procedure division.\n\n call cowtemplate using the-vars ""index.cow"".\n\n \n goback.\n\n end program indexweb.\n\n","\nSure, here's a commented version of the code:\n\n```\nidentification division.\nprogram-id. indexweb.\n\ndata division.\nworking-storage section.\n01 the-vars.\n03 COW-vars OCCURS 99 times.\n05 COW-varname pic x(99).\n05 COW-varvalue pic x(99).\n\nprocedure division.\ncall cowtemplate using the-vars ""index.cow"".\n\ngoback.\n\nend program indexweb.\n```\n\nThe code is divided into three sections: identification, data, and procedure.\n\nThe identification section contains the program name and ID.\n\nThe data section defines the working storage section, which is used to store data during program execution. The working storage section contains two segments: the-vars, which is a list of variables used by the program, and COW-vars, which is a list of variables used by the COW template.\n\nThe COW-vars segment is defined using the OCCURS clause, which specifies that it will repeat 99 times. Each time the OCCURS clause is encountered, the code will create a new instance of the COW-vars record. The COW-varname and COW-varvalue segments are used to define the name and value of each variable in the COW-vars record.\n\nThe procedure section contains the main program logic. The call statement calls the COW template and passes it the the-vars list. The goback statement returns control to the caller.\n\nThe end program statement ends the program."
2,"\n\n identification division.\n program-id. showname.\n\n data division.\n working-storage section.\n\n 01 the-vars.\n\n 03 COW-vars OCCURS 99 times.\n \n 05 COW-varname pic x(99).\n 05 COW-varvalue pic x(99). \n\n linkage section.\n\n 01 the-values.\n\n 05 COW-query-values occurs 10 times.\n 10 COW-query-value-name pic x(90).\n 10 COW-query-value pic x(90).\n\n\n procedure division using the-values.\n\n\n MOVE ""username"" to COW-varname(1).\n MOVE COW-query-value(1) to COW-varvalue(1). \n\n call cowtemplate using the-vars ""hello.cow"".\n\n \n goback.\n\n end program showname.\n\n","\nHere is the documentation for the code you provided:\n\nIdentification Division:\nThe identification division is where the program name and version number are specified.\n\n```\npr

In [49]:
df=pd.DataFrame(data)
df['index'] = df.index
df

,Code,Documentation,index
0,"move 3 to nroutes.\n\nmove to routing-pattern(1).\nmove indexweb to routing-destiny(1).\n\nmove /showsum/%value1/%value2"" to routing-pattern(2).\nmove ""showsum"" to routing-destiny(2).\n\nmove ""/showname/%value"" to routing-pattern(3).\nmove ""showname"" to routing-destiny(3).\n","\nSure, here's a well-detailed documentation of the code you provided:\n\nThe first line of code is a move command that moves a file from the user's Downloads folder to a folder named ""routing-pattern"" with the number 1.\nThe second line of code is a move command that moves a file from the user's Downloads folder to a folder named ""routing-destiny"" with the number 1.\nThe third line of code is a move command that moves a file from the user's Downloads folder to a folder named ""routing-pattern"" with the number 2.\nThe fourth line of code is a move command that moves a file from the user's Downloads folder to a folder named ""routing-destiny"" with the number 2.\nThe fifth line of code is a move command that moves a file from the user's Downloads folder to a folder named ""routing-pattern"" with the number 3.\nThe sixth line of code is a move command that moves a file from the user's Downloads folder to a folder named ""routing-destiny"" with the number 3.\n\nIn this way, the code creates three folders in the user's Downloads folder, each containing a file named ""showsum"" or ""showname"". The files are moved to these folders based on the values of the variables %value1, %value2, and %value3.",0
1,"\n\n identification division.\n program-id. indexweb.\n\n data division.\n working-storage section.\n\n 01 the-vars.\n\n 03 COW-vars OCCURS 99 times.\n \n 05 COW-varname pic x(99).\n 05 COW-varvalue pic x(99). \n\n\n procedure division.\n\n call cowtemplate using the-vars ""index.cow"".\n\n \n goback.\n\n end program indexweb.\n\n","\nSure, here's a commented version of the code:\n\n```\nidentification division.\nprogram-id. indexweb.\n\ndata division.\nworking-storage section.\n01 the-vars.\n03 COW-vars OCCURS 99 times.\n05 COW-varname pic x(99).\n05 COW-varvalue pic x(99).\n\nprocedure division.\ncall cowtemplate using the-vars ""index.cow"".\n\ngoback.\n\nend program indexweb.\n```\n\nThe code is divided into three sections: identification, data, and procedure.\n\nThe identification section contains the program name and ID.\n\nThe data section defines the working storage section, which is used to store data during program execution. The working storage section contains two segments: the-vars, which is a list of variables used by the program, and COW-vars, which is a list of variables used by the COW template.\n\nThe COW-vars segment is defined using the OCCURS clause, which specifies that it will repeat 99 times. Each time the OCCURS clause is encountered, the code will create a new instance of the COW-vars record. The COW-varname and COW-varvalue segments are used to define the name and value of each variable in the COW-vars record.\n\nThe procedure section contains the main program logic. The call statement calls the COW template and passes it the the-vars list. The goback statement returns control to the caller.\n\nThe end program statement ends the program.",1
2,"\n\n identification division.\n program-id. showname.\n\n data division.\n working-storage section.\n\n 01 the-vars.\n\n 03 COW-vars OCCURS 99 times.\n \n 05 COW-varname pic x(99).\n 05 COW-varvalue pic x(99). \n\n linkage section.\n\n 01 the-values.\n\n 05 COW-query-values occurs 10 times.\n 10 COW-query-value-name pic x(90).\n 10 COW-query-value pic x(90).\n\n\n procedure division using the-values.\n\n\n MOVE ""username"" to COW-varname(1).\n MOVE COW-query-value(1) to COW-varvalue(1). \n\n call cowtemplate using the-vars ""hello.cow"".\n\n \n goback.\n\n end program showname.\n\n","\nHere is the documentation for the code you provided:\n\nIdentification Division:\nThe identification division is where the program name and version number are specified.\

In [51]:
df.columns

Index(['Code', 'Documentation', 'index'], dtype='object')

In [15]:

import pandas as pd

# list of items
items = [{'name':'coffee','price':10.0}, {'name':'tea','price':5.0}, {'name':'breverage','price': 6.0}]

# empty the dataframe
price_frame = pd.DataFrame(columns = ['name','price'])

# iterating the items and append them to the dataframe
for item in items:
    price_frame = price_frame.append({'name':item['name'], 'price':item['price']}, ignore_index = True)

price_frame

<ipython-input-15-1acaa9baebbb>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  price_frame = price_frame.append({'name':item['name'], 'price':item['price']}, ignore_index = True)
<ipython-input-15-1acaa9baebbb>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  price_frame = price_frame.append({'name':item['name'], 'price':item['price']}, ignore_index = True)
<ipython-input-15-1acaa9baebbb>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  price_frame = price_frame.append({'name':item['name'], 'price':item['price']}, ignore_index = True)


,name,price
0,coffee,10.0
1,tea,5.0
2,breverage,6.0


In [30]:
output

'\nFlask is a micro web framework written in Python. It is classified as a microframework because it does not require particular tools or libraries. It has no database abstraction layer, form validation, or any other components where pre-existing third-party libraries provide common functions. However, Flask supports extensions that can add application features as if they were implemented in Flask itself.'

In [29]:
output=generate_response("What is Flask?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:399: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


In [47]:
generate_response("def scrape_url(url):")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:399: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


"\ndef scrape_url(url):\n    response = requests.get(url)\n    soup = BeautifulSoup(response.text, 'html.parser')\n    # extract data from soup and return it"

In [ ]:
generate_response("write a JS function that sorts a list alphabetically")

write a JS function that sorts a list alphabetically


function sortListAlphabetically(list) {
  list.sort();
}
